In [35]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump

INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows',100)

In [36]:
# read Square csv and pickle df
df1 = pd.read_csv(f'{INPUT_DIR}/items-2021-02-01-2021-10-02.csv')
df2 = pd.read_csv(f'{INPUT_DIR}/items-2021-09-06-2022-09-07.csv')
df = pd.concat([df1, df2])
df = df.drop_duplicates()  # drop overlap between the 2 csv files
# df.sort_values(by=['Date', 'Time'])
# df.reset_index(drop=True)  # create new index
df = df.sort_values(by=['Date', 'Time'])
df = df.reset_index(drop=True)  # create new index
pickle_dump(df, path_pickle_dump, 'df_sq_download')
print(f'df.shape: {df.shape}')

df.shape: (18366, 27)


In [ ]:
df

In [ ]:
df.columns

In [37]:
# Los Angeles temperature and precipitation data
# https://www.ncei.noaa.gov/cdo-web/datatools/findstation

df_weather = pd.read_csv(f'{INPUT_DIR}/la_weather_2021-01-01_2022-09-28.csv')
# df_weather = pd.read_csv(f'{INPUT_DIR}/la_temp.csv')

df_weather = df_weather.set_index("DATE")
# df_weather = df_weather.set_index("Date")

df_weather.index = pd.to_datetime(df_weather.index)  # change index to datetime before concat

In [ ]:
df_weather

In [ ]:
# clean data
df = pickle_load(path_pickle_dump, 'df_sq_download')
df_shape_before_clean = df.shape
# select only these columns
df = df[['Date', 'Time', 'Category', 'Item', 'Qty',
       'Price Point Name', 'Gross Sales',
       'Discounts', 'Net Sales', 'Tax', 
       'Device Name', 'Notes', 'Event Type', 
       'Dining Option', 'Customer Name']]
# strip leading $, convert from object to float
cols_to_clean = ['Gross Sales', 'Discounts', 'Net Sales', 'Tax']
for col in cols_to_clean:
  df[col] = df[col].str.split('$').str[-1]  # strip leading $
  df[col] = pd.to_numeric(df[col])  # convert from object to float
df = df.set_index('Date')  # set index to Date column
df.index = pd.to_datetime(df.index)  # convert Date string to datetime 
df.Time = pd.to_datetime(df.Time, format= '%H:%M:%S').dt.time  # convert Time string to hour:minute:second
# remove whole sale customer "Canyon Coffee" "Canyon Coffee"
print(f'df.shape before removing whole sale customer "Canyon Coffee": {df.shape}')
df = df[df['Customer Name'] != "Canyon Coffee"]
print(f'df.shape after removing whole sale customer "Canyon Coffee":  {df.shape}')
df_shape_after_clean = df.shape
pickle_dump(df, path_pickle_dump, 'df_clean')
print(f'df.shape before clean:  {df_shape_before_clean}')
print(f'df.shape after clean:   {df_shape_after_clean}')

In [ ]:
df = pickle_load(path_pickle_dump, 'df_clean')
print(f'df.shape: {df.shape}')

In [ ]:
print(f'Time of fist row, hour: {df.head(1).Time[0].hour}, minute: {df.head(1).Time[0].minute}, second: {df.head(1).Time[0].second}')

In [ ]:
# sum daily sales into a series
gross = df.groupby('Date')['Gross Sales'].sum()
# convert series into dataframe
df_gross = pd.DataFrame(gross)
df_gross.index = pd.to_datetime(df_gross.index)  # change index to datetime before concat

In [ ]:
# df_gross

In [ ]:
# Los Angeles temperature and precipitation data
# https://www.ncei.noaa.gov/cdo-web/datatools/findstation
df_weather = pd.read_csv(f'{INPUT_DIR}/la_temp.csv')
df_weather = df_weather.set_index("Date")
df_weather.index = pd.to_datetime(df_weather.index)  # change index to datetime before concat

In [ ]:
# concatenate columns of df_gross and df_weather using inner join
df_grSales_weather = pd.concat([df_gross, df_weather], axis=1, join='inner')
df_grSales_weather['DoW'] = \
  df_grSales_weather['DoW'].replace(to_replace=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'], value=[0, 1, 2, 3, 4, 5, 6])
pickle_dump(df_grSales_weather, path_pickle_dump, 'df_grSales_weather')

In [ ]:
df_grSales_weather = pickle_load(path_pickle_dump, 'df_grSales_weather')
df_grSales_weather